In [1]:
#import all the packages and libraries needed to execute the script
import numpy as np
import pandas as pd
import pickle
import pyabf
import tkinter as tk
import datetime

In [2]:
ZI_Spikes = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/IF_curve_Spp1-Slc30a3_06292022.csv')

# Functions --------------------------------------------------------------------------------------

In [ ]:
#KM 07-11-2022: These following Date/Neuron name functions might be unnecessary now that KM has updated neuronID in Intrinsic Property notebook

In [2]:
def reformat_Date(DF):
    
    """
        1 required argument:
            DF = dataframe with a 'Filepath' column
    
    """
    datelist = []
    DATELIST = []

    for item in DF['Filepath']:
       
        x = item
        
        if x.rfind('/KM/') != -1:
            a = x.rfind('/KM/')
            x = x[:a]
            
        elif x.rfind('/RK/') != -1:
            a = x.rfind('/RK/')
            x = x[:a]

        b = x.rfind('/')
        x = x[b+1:a]
        
        datelist.append(x)

    for idx,item in enumerate(datelist):
        if len(item) == 6:
            DATELIST.append(pd.to_datetime([item], format = '%m%d%y').date[0])
        elif len(item) == 8:
            DATELIST.append(pd.to_datetime([item], format = '%m%d%Y').date[0])
        else:
            DATELIST.append(idx)

    return DATELIST

In [3]:
def reformat_neuron_name(DF, NeuronColumn):
    
    """
        100221 RK wrote this before realizing that I had already written something similar in the IF_Spike_Counting Notebook
        I think this is a more compact code though, and is meant to convert the Neuron column of the compiled IF csv into 
        a consistent 5 character string composed of the experimenter initials and Neuron #
    """

    NewNeuronList = []

    for item in DF[NeuronColumn]:

        if type(item) == str: #only needed because there is an empty/Nan value at index 16
            
            if item.find('/') == -1:
                if item.find('RK') != -1:
                    NewNeuronList.append(item[:item.find('RK')+2] + "/" + item[item.find('RK'):])
                   

                elif item.find('KM') != -1:
                    NewNeuronList.append(item[:item.find('KM')+2] + "/" + item[item.find('KM'):])
            
                elif len(item) == 2:
                    NewNeuronList.append('KM/' + item)

        else:
            NewNeuronList.append(item)

            
    #Add an 'N' immediately after the slash
    for indx, val in enumerate(NewNeuronList):
        
        if type(val) == str:
            
            if val[val.find('/')+1] != 'N':

                NewNeuronList[indx] = val[:val.find('/')+1] + 'N' + val[val.find('/')+1:]
                
                
    #Limit the string to just the 5 critical characters (e.g. KM/N1)          
    for indx, val in enumerate(NewNeuronList):

        if type(val) == str:

            if val.find('/N') >= 0:

                new = val[:(val.find('/N'))+2] + [char for char in val if char.isdigit()][0]

                NewNeuronList[indx] = new

    return NewNeuronList

# Read in helpful lists and dictionaries

In [4]:
NeuronIDlist_andRegion = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/QualitativeData_Complete_07142022.csv', encoding='mac_roman')

NeuronIDlist = NeuronIDlist_andRegion['NeuronID']
NeuronRegionlist = NeuronIDlist_andRegion['Updated Region']

In [5]:
ID_Region = dict(zip(NeuronIDlist_andRegion['NeuronID'],NeuronIDlist_andRegion['Updated Region']))

In [6]:
ID_CellType = dict(zip(NeuronIDlist_andRegion['NeuronID'],NeuronIDlist_andRegion['CellType']))

# ------------------------------ TIDY UP AND REFORMAT DATA ----------------------------------

# IF DATA------------------------------------------------------------------------------------------

In [8]:
# Read In most updated version of DF to TIDY up
ZI_SpikesDF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/FinalVersions/2020-2022_IF_sweepspecific_07072022.csv')

C:\Users\Ryan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3325: DtypeWarning: Columns (1,2,3,8,16,23,25,26,27,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
ZI_SpikesDF

,Unnamed: 0,Other Observations,Spiking Pattern,Region,Cell Type,Date,Neuron,NeuronID,File,Sweep,...,Rise Time 10% to 90% (ms),Decay Slope,Decay Time 90% o 10% (ms),Area(mV*ms),Inst. Freq. (Hz),Interevent Interval (ms),S.D. of Fit,Template Match,Re-arm (mV),Trigger (mV)
0,124,"Depolarization induced adaptation, Spontaneous...",Fast Spiking,NaN,Spp1-Cre,9/22/2020,RK/N1,2020-09-22/RK/N1,20922002.abf,1,...,Not found,Not found,Not found,33.2214241,33.73819351,29.63999939,NaN,NaN,NaN,NaN
1,123,"Depolarization induced adaptation, Spontaneous...",Fast Spiking,NaN,Spp1-Cre,9/22/2020,RK/N1,2020-09-22/RK/N1,20922002.abf,1,...,Not found,Not found,Not found,33.4165535,NaN,Not found,NaN,NaN,NaN,NaN
2,133,"Depolarization induced adaptation, Spontaneous...",Fast Spiking,NaN,Spp1-Cre,9/22/2020,RK/N1,2020-09-22/RK/N1,20922002.abf,2,...,Not found,Not found,Not found,34.09707642,65.61637115,15.24009991,NaN,NaN,NaN,NaN
3,132,"Depolarization induced adaptation, Spontaneous...",Fast Spiking,NaN,Spp1-Cre,9/22/2020,RK/N1,2020-09-22/RK/N1,20922002.abf,2,...,Not found,Not found,Not found,33.68122864,59.80932617,16.71980095,NaN,NaN,NaN,NaN
4,131,"Depolarization induced adaptation, Spontaneous...",Fast Spiking,NaN,Spp1-Cre,9/22/2020,RK/N1,2020-09-22/RK/N1,20922002.abf,2,...,Not found,Not found,Not found,34.00993347,60.97472,16.40023994,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56608,4229,NaN,NaN,NaN,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,Not found,Not found,Not found,565.910584,97.080017,10.30078125,NaN,NaN,NaN,NaN
56609,4228,NaN,NaN,NaN,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,Not found,Not found,Not found,558.381103,97.117607,10.29679394,NaN,NaN,NaN,NaN
56610,4226,NaN,NaN,NaN,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,Not found,Not found,Not found,565.880859,100.510407,9.94921875,NaN,NaN,NaN,NaN
56611,4255,NaN,NaN,NaN,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,Not found,Not found,Not found,576.527771,80.961952,12.35148144,NaN,NaN,NaN,NaN


In [47]:
ZI_SpikesDF =  ZI_SpikesDF.astype({'Date':'datetime64[ns]'})

In [49]:
NID_list = []

for i in range(len(ZI_SpikesDF)):
    temp = str(ZI_SpikesDF.iloc[i]['Date']) + '/' + ZI_SpikesDF.iloc[i]['Experimenter'] + '/' +  ZI_SpikesDF.iloc[i]['Neuron']
    NID_list.append(temp)

ZI_SpikesDF['NeuronID'] = NID_list

In [15]:
set(ZI_SpikesDF['NeuronID'])

{'2020-09-22/RK/N1',
 '2020-09-22/RK/N2',
 '2020-09-22/RK/N3',
 '2020-09-22/RK/N5',
 '2020-09-22/RK/N6',
 '2020-09-23/KM-N1',
 '2020-09-23/KM/N2',
 '2020-09-23/KM/N3',
 '2020-09-23/KM/N4',
 '2020-09-23/RK/N1',
 '2020-09-23/RK/N2',
 '2020-09-23/RK/N3',
 '2020-09-23/RK/N4',
 '2020-09-23/RK/N5',
 '2020-09-23/RK/N6',
 '2020-09-24/KM-N1',
 '2020-09-24/KM/N2',
 '2020-09-24/KM/N3',
 '2020-09-24/KM/N4',
 '2020-09-24/KM/N5',
 '2020-09-24/KM/N6',
 '2020-09-25/RK/N1',
 '2020-10-01/KM/N1',
 '2020-10-01/KM/N2',
 '2020-10-01/KM/N3',
 '2020-10-01/KM/N4',
 '2020-10-01/KM/N5',
 '2020-10-01/KM/N6',
 '2020-10-01/RK/N2',
 '2020-10-01/RK/N3',
 '2020-10-01/RK/N4',
 '2020-10-01/RK/N5',
 '2020-10-01/RK/N6',
 '2020-10-02/KM/N1',
 '2020-10-02/KM/N2',
 '2020-10-02/KM/N3',
 '2020-10-02/KM/N4',
 '2020-10-02/KM/N5',
 '2020-10-02/RK/N1',
 '2020-10-02/RK/N2',
 '2020-10-02/RK/N4',
 '2021-06-15/KM/N1',
 '2021-06-17/KM/N1',
 '2021-06-17/KM/N2',
 '2021-06-17/KM/N3',
 '2021-06-21/KM/N1',
 '2021-06-21/KM/N2',
 '2021-06-21/

In [14]:
ZI_SpikesDF = ZI_SpikesDF.replace("2020-09-23/KM-N1'","2020-09-23/KM-N1")

In [23]:
ID_CellType

{'2020-09-22/RK/N2': 'Spp1',
 '2020-09-22/RK/N3': 'Spp1',
 '2020-09-22/RK/N5': 'Spp1',
 '2020-09-23/KM/N1': 'Spp1',
 '2020-09-23/KM/N2': 'Spp1',
 '2020-09-23/KM/N3': 'Spp1',
 '2020-09-23/RK/N1': 'Spp1',
 '2020-09-23/RK/N2': 'Spp1',
 '2020-09-23/RK/N3': 'Spp1',
 '2020-09-23/RK/N4': 'Spp1',
 '2020-09-23/RK/N5': 'Spp1',
 '2020-09-23/RK/N6': 'Spp1',
 '2020-09-24/KM/N1': 'Spp1',
 '2020-09-24/KM/N2': 'Spp1',
 '2020-09-24/KM/N3': 'Spp1',
 '2020-09-24/KM/N4': 'Spp1',
 '2020-09-24/KM/N5': 'Spp1',
 '2020-09-24/KM/N6': 'Spp1',
 '2020-10-01/KM/N1': 'Nos1',
 '2020-10-01/KM/N2': 'Nos1',
 '2020-10-01/KM/N3': 'Nos1',
 '2020-10-01/KM/N4': 'Nos1',
 '2020-10-01/KM/N5': 'Nos1',
 '2020-10-01/KM/N6': 'Nos1',
 '2020-10-01/RK/N1': 'Nos1',
 '2020-10-01/RK/N2': 'Nos1',
 '2020-10-01/RK/N3': 'Nos1',
 '2020-10-01/RK/N4': 'Nos1',
 '2020-10-01/RK/N5': 'Nos1',
 '2020-10-01/RK/N6': 'Nos1',
 '2020-10-02/KM/N1': 'Nos1',
 '2020-10-02/KM/N2': 'Nos1',
 '2020-10-02/KM/N3': 'Nos1',
 '2020-10-02/KM/N4': 'Nos1',
 '2020-10-02/K

In [16]:
#Given that some of the measures in the IF csvs end up with NaN values, we have to convert these columns to 
#numeric values using pd.to_numeric()
for item in ['Half-width (ms)',
             'Time to Rise Half-amplitude', 
             'Time to Decay Half-amplitude',
             'Rise Tau (ms)', 
             'Rise Slope 10% to 90% (mV/ms)', 
             'Rise Time 10% to 90% (ms)', 
             'Decay Slope', 
             'Decay Time 90% o 10% (ms)', 
             'Interevent Interval (ms)',
             'Decay Tau (ms)',
             'Max Rise slope (mV/ms)',
             'Time to Max Rise slope (ms)']:

    ZI_SpikesDF[item] = pd.to_numeric(ZI_SpikesDF[item], errors = 'coerce')

In [58]:
### Use the ID_Region dictionary to assign regions to the ZI_SpikesDF['Region'] column

regionlist = []

for i in range(len(ZI_SpikesDF)):
    
    if ZI_SpikesDF.iloc[i]['NeuronID'] in list(ID_Region.keys()):
        
        reg = ID_Region[ZI_SpikesDF.iloc[i]['NeuronID']]
        regionlist.append(reg)
    else:
        regionlist.append(ZI_SpikesDF.iloc[i]['Trigger (mV)'])
    
ZI_SpikesDF['Region'] = regionlist
        

In [59]:
### Use the ID_CellType dictionary to assign cell type to the ZI_SpikesDF['Cell Type'] column

celltypelist = []

for i in range(len(ZI_SpikesDF)):
    
    if ZI_SpikesDF.iloc[i]['NeuronID'] in list(ID_CellType.keys()):
        
        ct = ID_CellType[ZI_SpikesDF.iloc[i]['NeuronID']]
        celltypelist.append(ct)
    else:
        celltypelist.append(ZI_SpikesDF.iloc[i]['Trigger (mV)'])
    
ZI_SpikesDF['Cell Type'] = celltypelist
        

In [47]:
for i in range(len(ZI_SpikesDF)):
    
    if ZI_SpikesDF.iloc[i]['NeuronID'] not in list(ID_CellType.keys()):
        ZI_SpikesDF.drop(i)

In [65]:
ZI_SpikesDF = ZI_SpikesDF[ZI_SpikesDF["Region"].str.contains("NaN") == False]
  
ZI_SpikesDF

,Unnamed: 0,Other Observations,Spiking Pattern,Region,Cell Type,Date,Neuron,NeuronID,File,Sweep,...,Rise Time 10% to 90% (ms),Decay Slope,Decay Time 90% o 10% (ms),Area(mV*ms),Inst. Freq. (Hz),Interevent Interval (ms),S.D. of Fit,Template Match,Re-arm (mV),Trigger (mV)
311,1204,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.239770,-201.146927,0.281163,5.426757813,NaN,NaN,NaN,NaN,NaN,NaN
312,1219,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.261159,-193.631561,0.287479,9.854539871,13.40482616,74.599998,NaN,NaN,NaN,NaN
313,1217,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,NaN,-191.982987,0.288354,29.91509247,12.1772871,82.120102,NaN,NaN,NaN,NaN
314,1216,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.265003,-191.982987,0.288354,9.921269417,12.17724419,82.120392,NaN,NaN,NaN,NaN
315,1215,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,NaN,-192.953323,0.285192,29.96653366,11.64959908,85.839859,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56608,4229,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,565.910584,97.080017,10.300781,NaN,NaN,NaN,NaN
56609,4228,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,558.381103,97.117607,10.296794,NaN,NaN,NaN,NaN
56610,4226,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,565.880859,100.510407,9.949219,NaN,NaN,NaN,NaN
56611,4255,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,576.527771,80.961952,12.351481,NaN,NaN,NaN,NaN


In [52]:
ZI_SpikesDF = ZI_SpikesDF[~ZI_SpikesDF.NeuronID.str.contains('2020-09-23/KM/N1')]
ZI_SpikesDF

,Unnamed: 0,Other Observations,Spiking Pattern,Region,Cell Type,Date,Neuron,NeuronID,File,Sweep,...,Rise Time 10% to 90% (ms),Decay Slope,Decay Time 90% o 10% (ms),Area(mV*ms),Inst. Freq. (Hz),Interevent Interval (ms),S.D. of Fit,Template Match,Re-arm (mV),Trigger (mV)
311,1204,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.239770,-201.146927,0.281163,5.426757813,NaN,NaN,NaN,NaN,NaN,NaN
312,1219,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.261159,-193.631561,0.287479,9.854539871,13.40482616,74.599998,NaN,NaN,NaN,NaN
313,1217,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,NaN,-191.982987,0.288354,29.91509247,12.1772871,82.120102,NaN,NaN,NaN,NaN
314,1216,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.265003,-191.982987,0.288354,9.921269417,12.17724419,82.120392,NaN,NaN,NaN,NaN
315,1215,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,NaN,-192.953323,0.285192,29.96653366,11.64959908,85.839859,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56608,4229,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,565.910584,97.080017,10.300781,NaN,NaN,NaN,NaN
56609,4228,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,558.381103,97.117607,10.296794,NaN,NaN,NaN,NaN
56610,4226,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,565.880859,100.510407,9.949219,NaN,NaN,NaN,NaN
56611,4255,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,576.527771,80.961952,12.351481,NaN,NaN,NaN,NaN


In [72]:
ZI_SpikesDF = ZI_SpikesDF.drop('Unnamed: 0',axis=1)

In [79]:
ZI_SpikesDF['Region_CellType'] = ZI_SpikesDF[['Region','Cell Type']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


In [80]:
ZI_SpikesDF['Region_CellType'].value_counts()

dZI Slc30a3             2123
vZI Spp1                1196
d/vZI Border Slc30a3     844
vZI Slc30a3               94
Name: Region_CellType, dtype: int64

In [35]:
ZI_SpikesDF = ZI_SpikesDF[~ZI_SpikesDF.NeuronID.str.contains('2020-09-22/RK/N1')]

In [39]:
ZI_SpikesDF

,Unnamed: 0,Other Observations,Spiking Pattern,Region,Cell Type,Date,Neuron,NeuronID,File,Sweep,...,Rise Time 10% to 90% (ms),Decay Slope,Decay Time 90% o 10% (ms),Area(mV*ms),Inst. Freq. (Hz),Interevent Interval (ms),S.D. of Fit,Template Match,Re-arm (mV),Trigger (mV)
311,1204,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.239770,-201.146927,0.281163,5.426757813,NaN,NaN,NaN,NaN,NaN,NaN
312,1219,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.261159,-193.631561,0.287479,9.854539871,13.40482616,74.599998,NaN,NaN,NaN,NaN
313,1217,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,NaN,-191.982987,0.288354,29.91509247,12.1772871,82.120102,NaN,NaN,NaN,NaN
314,1216,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,0.265003,-191.982987,0.288354,9.921269417,12.17724419,82.120392,NaN,NaN,NaN,NaN
315,1215,"Rebound spiking, Spontaneous Firing","Fast Spiking, No Hz Adaptation",FF,Spp1,9/22/2020,RK/N2,2020-09-22/RK/N2,20922004.abf,2,...,NaN,-192.953323,0.285192,29.96653366,11.64959908,85.839859,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56608,4229,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,565.910584,97.080017,10.300781,NaN,NaN,NaN,NaN
56609,4228,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,558.381103,97.117607,10.296794,NaN,NaN,NaN,NaN
56610,4226,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,565.880859,100.510407,9.949219,NaN,NaN,NaN,NaN
56611,4255,NaN,NaN,dZI,Slc30a3,6/17/2022,N2,2022-06-17/RK/N2,2022_06_17_0007.abf,16,...,NaN,NaN,NaN,576.527771,80.961952,12.351481,NaN,NaN,NaN,NaN


In [74]:
ZI_SpikesDF.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/FinalVersions/2020-2022_IF_sweepspecific_07102022.csv')

# Ih Data------------------------------------------------------------------------------------------

In [75]:
# Read In most updated version of DF to TIDY up
ZI_Ih_DF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/FinalVersions/2020-2022_Ih_07082022.csv')


In [76]:
ZI_Ih_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           828 non-null    int64  
 1   level_0                              828 non-null    int64  
 2   index                                828 non-null    int64  
 3   Cell Type                            828 non-null    object 
 4   Date                                 828 non-null    object 
 5   Mouse                                828 non-null    object 
 6   Neuron                               828 non-null    object 
 7   Filename                             828 non-null    object 
 8   Protocol                             828 non-null    object 
 9   Filepath                             828 non-null    object 
 10  Sweep Number                         828 non-null    int64  
 11  Holding Voltage (mV)            

In [97]:
list(ZI_Ih_DF['NeuronID'].unique())

['2022-06-13/KM/N1',
 '2022-06-13/KM/N2',
 '2022-06-14/KM/N1',
 '2022-06-14/KM/N2',
 '2022-06-14/KM/N3',
 '2022-06-14/KM/N4',
 '2022-06-15/KM/N1',
 '2022-06-15/KM/N2',
 '2022-06-15/KM/N3',
 '2022-06-15/RK/N1',
 '2022-06-15/RK/N2',
 '2022-06-17/KM/N1',
 '2022-06-17/KM/N2',
 '2022-06-17/KM/N3',
 '2022-06-17/KM/N4',
 '2022-06-17/KM/N5']

In [79]:
### Use the ID_Region dictionary to assign regions to the ZI_Ih_DF['Region'] column

regionlist = []

for i in range(len(ZI_Ih_DF)):
    
    if ZI_Ih_DF.iloc[i]['NeuronID'] in list(ID_Region.keys()):
        
        reg = ID_Region[ZI_Ih_DF.iloc[i]['NeuronID']]
        regionlist.append(reg)
    else:
        regionlist.append(ZI_Ih_DF.iloc[i]['NeuronID'])
    
ZI_Ih_DF['Region'] = regionlist
        

In [81]:
### Use the ID_CellType dictionary to assign cell type to the ZI_Ih_DF['Cell Type'] column

celltypelist = []

for i in range(len(ZI_Ih_DF)):
    
    if ZI_Ih_DF.iloc[i]['NeuronID'] in list(ID_CellType.keys()):
        
        ct = ID_CellType[ZI_Ih_DF.iloc[i]['NeuronID']]
        celltypelist.append(ct)
    else:
        celltypelist.append(ZI_Ih_DF.iloc[i]['NeuronID'])
    
ZI_Ih_DF['Cell Type'] = celltypelist
        

In [82]:
ZI_Ih_DF

,Unnamed: 0,level_0,index,Cell Type,Date,Mouse,Neuron,Filename,Protocol,Filepath,...,Holding Voltage (mV),Voltage Step (mV),Ih_Ra (MOhms),Holding Current (pA),Total Current (pA),Ih (pA),Tail Current Peak (pA),Baseline Adjusted Tail Current (pA),NeuronID,Region
0,0,258,0,Spp1,2020-09-22,,RK/N3,20922009.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-70.0,15.421810,-17.766418,-48.794860,-20.479736,-32.348633,-14.143995,2020-09-22/RK/N3,vZI
1,1,284,8,Spp1,2020-09-22,,RK/N5,20922012.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-150.0,16.774395,-5.468140,-411.319000,-27.993195,-25.329590,-14.518063,2020-09-22/RK/N5,vZI
2,2,297,0,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-70.0,17.282797,20.382566,-36.796627,-20.442380,6.225586,-8.180857,2020-09-22/RK/N2,FF
3,3,298,1,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-80.0,17.071128,13.854002,-83.496820,-31.923092,-13.305663,-16.117529,2020-09-22/RK/N2,FF
4,4,299,2,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-90.0,17.040721,5.606201,-142.944810,-50.546623,-32.470700,-18.403473,2020-09-22/RK/N2,FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,836,1129,2,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,...,-60.0,-90.0,13.200553,155.078720,-243.227260,-12.464966,132.324200,-16.820710,2022-06-17/KM/N3,dZI
824,837,1130,3,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,...,-60.0,-100.0,13.270925,153.512440,-317.389620,-11.167236,134.765610,-8.812195,2022-06-17/KM/N3,dZI
825,838,1131,4,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,...,-60.0,-110.0,13.299523,143.810970,-389.872740,-14.372314,123.168940,-16.284690,2022-06-17/KM/N3,dZI
826,839,1132,5,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,...,-60.0,-120.0,14.482281,142.993880,-465.459720,-17.412140,126.708980,-9.657829,2022-06-17/KM/N3,dZI


In [95]:
set(ZI_Ih_DF['Region'])

{'Ambiguous',
 'FF',
 'PIL',
 'SNR',
 'SNr',
 'STN',
 'cZI',
 'd/vZI Border',
 'dZI',
 'vLGN',
 'vZI'}

In [94]:
ZI_Ih_DF = ZI_Ih_DF[~ZI_Ih_DF.NeuronID.str.contains('2020-09-23/yy.ab')]
ZI_Ih_DF = ZI_Ih_DF[~ZI_Ih_DF.NeuronID.str.contains('2020-09-24/_y.ab')]
ZI_Ih_DF = ZI_Ih_DF[~ZI_Ih_DF.NeuronID.str.contains('2020-09-24/yy.ab')]
ZI_Ih_DF = ZI_Ih_DF[~ZI_Ih_DF.NeuronID.str.contains('2021-06-18/KM/N2')]

ZI_Ih_DF

,Unnamed: 0,level_0,index,Cell Type,Date,Mouse,Neuron,Filename,Protocol,Filepath,...,Holding Voltage (mV),Voltage Step (mV),Ih_Ra (MOhms),Holding Current (pA),Total Current (pA),Ih (pA),Tail Current Peak (pA),Baseline Adjusted Tail Current (pA),NeuronID,Region
0,0,258,0,Spp1,2020-09-22,,RK/N3,20922009.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-70.0,15.421810,-17.766418,-48.794860,-20.479736,-32.348633,-14.143995,2020-09-22/RK/N3,vZI
1,1,284,8,Spp1,2020-09-22,,RK/N5,20922012.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-150.0,16.774395,-5.468140,-411.319000,-27.993195,-25.329590,-14.518063,2020-09-22/RK/N5,vZI
2,2,297,0,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-70.0,17.282797,20.382566,-36.796627,-20.442380,6.225586,-8.180857,2020-09-22/RK/N2,FF
3,3,298,1,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-80.0,17.071128,13.854002,-83.496820,-31.923092,-13.305663,-16.117529,2020-09-22/RK/N2,FF
4,4,299,2,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,...,-60.0,-90.0,17.040721,5.606201,-142.944810,-50.546623,-32.470700,-18.403473,2020-09-22/RK/N2,FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,836,1129,2,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,...,-60.0,-90.0,13.200553,155.078720,-243.227260,-12.464966,132.324200,-16.820710,2022-06-17/KM/N3,dZI
824,837,1130,3,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,...,-60.0,-100.0,13.270925,153.512440,-317.389620,-11.167236,134.765610,-8.812195,2022-06-17/KM/N3,dZI
825,838,1131,4,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,...,-60.0,-110.0,13.299523,143.810970,-389.872740,-14.372314,123.168940,-16.284690,2022-06-17/KM/N3,dZI
826,839,1132,5,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,...,-60.0,-120.0,14.482281,142.993880,-465.459720,-17.412140,126.708980,-9.657829,2022-06-17/KM/N3,dZI


In [97]:
ZI_Ih_DF = ZI_Ih_DF.sort_values('NeuronID')

In [104]:
ZI_Ih_DF = ZI_Ih_DF.drop('index',axis=1)
ZI_Ih_DF

,Cell Type,Date,Mouse,Neuron,Filename,Protocol,Filepath,Sweep Number,Holding Voltage (mV),Voltage Step (mV),Ih_Ra (MOhms),Holding Current (pA),Total Current (pA),Ih (pA),Tail Current Peak (pA),Baseline Adjusted Tail Current (pA),NeuronID,Region
0,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,9,-60.0,-150.0,22.881329,-66.661980,-729.622600,-138.325680,-132.568340,-39.579323,2020-09-22/RK/N2,FF
1,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,1,-60.0,-70.0,17.282797,20.382566,-36.796627,-20.442380,6.225586,-8.180857,2020-09-22/RK/N2,FF
2,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,2,-60.0,-80.0,17.071128,13.854002,-83.496820,-31.923092,-13.305663,-16.117529,2020-09-22/RK/N2,FF
3,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,3,-60.0,-90.0,17.040721,5.606201,-142.944810,-50.546623,-32.470700,-18.403473,2020-09-22/RK/N2,FF
4,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,4,-60.0,-100.0,17.003324,-7.381835,-223.436020,-81.647950,-65.673820,-33.373478,2020-09-22/RK/N2,FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,Slc30a3,2022-06-17,,KM/N5,22617040.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,5,-60.0,-110.0,13.302535,49.331780,-160.831880,-17.383904,19.653318,-21.543108,2022-06-17/KM/N5,dZI
786,Slc30a3,2022-06-17,,KM/N5,22617040.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,2,-60.0,-80.0,13.411538,56.805233,-59.574398,-10.337523,30.761717,-23.551435,2022-06-17/KM/N5,dZI
787,Slc30a3,2022-06-17,,KM/N5,22617040.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,4,-60.0,-100.0,13.364356,51.630184,-117.744690,-11.915287,10.742187,-33.942043,2022-06-17/KM/N5,dZI
788,Slc30a3,2022-06-17,,KM/N5,22617040.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,14.830443,45.732418,-195.855070,-16.089462,21.118162,-18.067327,2022-06-17/KM/N5,dZI


In [105]:
ZI_Ih_DF.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/FinalVersions/2020-2022_Ih_07102022.csv')

In [106]:
NeuronLevelIh = ZI_Ih_DF[ZI_Ih_DF['Sweep Number'] == 6]

In [107]:
len(NeuronLevelIh['NeuronID'])

115

In [108]:
NeuronLevelIh = NeuronLevelIh.drop_duplicates('NeuronID')
NeuronLevelIh

,Cell Type,Date,Mouse,Neuron,Filename,Protocol,Filepath,Sweep Number,Holding Voltage (mV),Voltage Step (mV),Ih_Ra (MOhms),Holding Current (pA),Total Current (pA),Ih (pA),Tail Current Peak (pA),Baseline Adjusted Tail Current (pA),NeuronID,Region
6,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,16.989660,-34.968258,-426.75067,-113.940640,-99.975580,-35.452750,2020-09-22/RK/N2,FF
10,Spp1,2020-09-22,,RK/N3,20922009.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,15.607772,-42.480164,-666.11847,-47.893066,-75.073240,-19.805523,2020-09-22/RK/N3,vZI
21,Spp1,2020-09-22,,RK/N5,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,23.059733,-33.559875,-253.98712,-25.706177,-47.302246,-8.993031,2020-09-22/RK/N5,vZI
43,Spp1,2020-09-23,,KM/N2,20923013.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,14.170464,-25.484314,-345.72050,-72.742890,-65.307620,-26.285892,2020-09-23/KM/N2,vZI
50,Spp1,2020-09-23,,KM/N3,20923020.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,8.817058,135.679630,-1714.34840,-279.230220,-15.258789,-58.216145,2020-09-23/KM/N3,vLGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,Slc30a3,2022-06-17,,KM/N1,22617004.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,19.918299,-14.305785,-174.53856,-11.598267,-20.996092,-3.136486,2022-06-17/KM/N1,dZI
768,Slc30a3,2022-06-17,,KM/N2,22617013.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,19.305768,0.579895,-87.10296,-7.421509,-8.300780,-8.384951,2022-06-17/KM/N2,d/vZI Border
777,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,14.482281,142.993880,-465.45972,-17.412140,126.708980,-9.657829,2022-06-17/KM/N3,dZI
780,Slc30a3,2022-06-17,,KM/N4,22617031.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,15.076895,-20.400572,-294.92530,-20.281952,-68.969720,-38.261850,2022-06-17/KM/N4,dZI


In [111]:
NeuronLevelIh = NeuronLevelIh.reset_index().drop('index',axis=1)

In [112]:
NeuronLevelIh

,Cell Type,Date,Mouse,Neuron,Filename,Protocol,Filepath,Sweep Number,Holding Voltage (mV),Voltage Step (mV),Ih_Ra (MOhms),Holding Current (pA),Total Current (pA),Ih (pA),Tail Current Peak (pA),Baseline Adjusted Tail Current (pA),NeuronID,Region
0,Spp1,2020-09-22,,RK/N2,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,16.989660,-34.968258,-426.75067,-113.940640,-99.975580,-35.452750,2020-09-22/RK/N2,FF
1,Spp1,2020-09-22,,RK/N3,20922009.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,15.607772,-42.480164,-666.11847,-47.893066,-75.073240,-19.805523,2020-09-22/RK/N3,vZI
2,Spp1,2020-09-22,,RK/N5,20922006.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,23.059733,-33.559875,-253.98712,-25.706177,-47.302246,-8.993031,2020-09-22/RK/N5,vZI
3,Spp1,2020-09-23,,KM/N2,20923013.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,14.170464,-25.484314,-345.72050,-72.742890,-65.307620,-26.285892,2020-09-23/KM/N2,vZI
4,Spp1,2020-09-23,,KM/N3,20923020.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,6,-60.0,-120.0,8.817058,135.679630,-1714.34840,-279.230220,-15.258789,-58.216145,2020-09-23/KM/N3,vLGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Slc30a3,2022-06-17,,KM/N1,22617004.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,19.918299,-14.305785,-174.53856,-11.598267,-20.996092,-3.136486,2022-06-17/KM/N1,dZI
98,Slc30a3,2022-06-17,,KM/N2,22617013.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,19.305768,0.579895,-87.10296,-7.421509,-8.300780,-8.384951,2022-06-17/KM/N2,d/vZI Border
99,Slc30a3,2022-06-17,,KM/N3,22617022.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,14.482281,142.993880,-465.45972,-17.412140,126.708980,-9.657829,2022-06-17/KM/N3,dZI
100,Slc30a3,2022-06-17,,KM/N4,22617031.abf,VC episodic (ih),/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,6,-60.0,-120.0,15.076895,-20.400572,-294.92530,-20.281952,-68.969720,-38.261850,2022-06-17/KM/N4,dZI


In [114]:
NeuronLevelIh.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_Ih_07102022.csv')

# Capacitance Data------------------------------------------------------------------------------------------

In [124]:
# Read In most updated version of DF to TIDY up
Cap_startDF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/FinalVersions/2020-2022_Cp_sweepspecific_07102022.csv')

In [125]:
Cap_startDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1897 entries, 0 to 1896
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               1897 non-null   int64  
 1   Unnamed: 0.1             1897 non-null   int64  
 2   Date                     1897 non-null   object 
 3   Neuron                   1897 non-null   object 
 4   NeuronID                 1897 non-null   object 
 5   FileName                 1897 non-null   object 
 6   Signal                   1897 non-null   object 
 7   Trace                    1897 non-null   int64  
 8   Trace start              1897 non-null   int64  
 9   Fit function             1897 non-null   object 
 10  Best number              0 non-null      float64
 11  A1                       1897 non-null   float64
 12  S.E.                     1897 non-null   float64
 13  Tau1                     1897 non-null   float64
 14  S.E..1                  

In [127]:
Cap_startDF.drop(['Unnamed: 0.1'], axis=1, inplace = True)

In [128]:
Cap_startDF

,Date,Neuron,NeuronID,FileName,Signal,Trace,Trace start,Fit function,Best number,A1,...,Correlation coefficient,Standard deviation,Sum of squared error,Sum of data squared,Points fitted,Fitted area,Results,File path,Cell Type,Capacitance (pF)
0,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-172.285049,...,0.9776631,4.239509583,8986.720703,18011766.0,501,9140.280273,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,90.944294
1,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,2,1000,"Exponential, standard, 2nd Order",NaN,-176.273025,...,0.977986038,4.248130322,9023.304688,18619554.0,501,9308.932617,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,86.996545
2,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,3,2000,"Exponential, standard, 2nd Order",NaN,-177.974960,...,0.963022351,7.088396072,25122.68164,18373332.0,501,9246.724609,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,80.815497
3,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,4,3000,"Exponential, standard, 2nd Order",NaN,-175.414535,...,0.978542447,4.202992916,8832.575195,19183462.0,501,9447.616211,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,88.345520
4,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,5,4000,"Exponential, standard, 2nd Order",NaN,-185.801666,...,0.978297114,4.348496437,9454.709961,20073086.0,501,9661.617188,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,95.655040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,6/17/2022,RK/N2,2022-06-17/RK/N2,2022_06_17_0006.abf,IN_0,16,3750,"Exponential, standard, 2nd Order",NaN,-153.385452,...,0.944687903,3.766649723,14187.65039,4637358.0,1001,2599.469482,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,58.778454
1893,6/17/2022,RK/N2,2022-06-17/RK/N2,2022_06_17_0006.abf,IN_0,17,4000,"Exponential, standard, 2nd Order",NaN,-121.140343,...,0.939854622,4.185803413,17520.95117,4843433.5,1001,2651.708984,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,70.646057
1894,6/17/2022,RK/N2,2022-06-17/RK/N2,2022_06_17_0006.abf,IN_0,18,4250,"Exponential, standard, 2nd Order",NaN,-131.036804,...,0.925960839,5.164000034,26666.89453,4864624.5,1001,2731.665527,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,62.076161
1895,6/17/2022,RK/N2,2022-06-17/RK/N2,2022_06_17_0006.abf,IN_0,19,4500,"Exponential, standard, 2nd Order",NaN,-424.984985,...,0.921567798,5.516321182,30429.79883,4946642.0,1001,2696.283447,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,56.271429


In [129]:
list(Cap_startDF['NeuronID'].unique())

['2020-09-22/RK/N2',
 '2020-09-22/RK/N3',
 '2020-09-22/RK/N5',
 '2020-09-23/RK/N1',
 '2020-09-23/RK/N2',
 '2020-09-23/RK/N3',
 '2020-09-23/RK/N4',
 '2020-09-23/RK/N5',
 '2020-09-23/RK/N6',
 '2020-10-01/RK/N1',
 '2020-10-01/RK/N2',
 '2020-10-01/RK/N3',
 '2020-10-01/RK/N4',
 '2020-10-01/RK/N5',
 '2020-10-01/RK/N6',
 '2020-10-02/RK/N1',
 '2020-10-02/RK/N2',
 '2020-10-02/RK/N3',
 '2021-06-15/KM/N1',
 '2021-06-17/KM/N1',
 '2021-06-17/KM/N2',
 '2021-06-17/KM/N3',
 '2021-06-21/KM/N1',
 '2021-06-21/KM/N2',
 '2021-06-21/KM/N3',
 '2021-06-21/KM/N4',
 '2021-06-21/KM/N5',
 '2021-06-21/KM/N6',
 '2021-06-22/KM/N2',
 '2021-06-22/KM/N3',
 '2021-06-22/KM/N4',
 '2021-06-28/KM/N1',
 '2021-06-28/KM/N3',
 '2021-06-28/KM/N4',
 '2021-06-28/KM/N5',
 '2021-06-29/KM/N1',
 '2021-06-29/KM/N2',
 '2021-06-29/KM/N3',
 '2021-06-30/KM/N1',
 '2021-06-30/KM/N2',
 '2021-06-30/KM/N3',
 '2021-07-01/KM/N1',
 '2021-07-01/KM/N2',
 '2021-07-01/KM/N4',
 '2021-08-04/KM/N1',
 '2021-08-04/KM/N2',
 '2021-08-04/KM/N3',
 '2021-08-05/

In [130]:
def median_Cp(DF):

#associate contiguous rows with the same string in filename column
    same_file = DF['FileName'].ne(DF['FileName'].shift()).cumsum()

    #Populate the 'Group' column using the same_file variable
    DF['Group'] = same_file
    
    medianCp = DF.groupby('NeuronID')['Capacitance (pF)'].median()
    medianCpList = []
   

    for index, row in DF.iterrows():
        x = row['NeuronID']
        medianCpList.append(medianCp[x]) 

    DF['Median Capacitance (pF)'] = medianCpList
 
    return DF

In [131]:
Med_Cp = median_Cp(Cap_startDF)

Med_Cp

,Date,Neuron,NeuronID,FileName,Signal,Trace,Trace start,Fit function,Best number,A1,...,Sum of squared error,Sum of data squared,Points fitted,Fitted area,Results,File path,Cell Type,Capacitance (pF),Group,Median Capacitance (pF)
0,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-172.285049,...,8986.720703,18011766.0,501,9140.280273,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,90.944294,1,85.325031
1,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,2,1000,"Exponential, standard, 2nd Order",NaN,-176.273025,...,9023.304688,18619554.0,501,9308.932617,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,86.996545,1,85.325031
2,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,3,2000,"Exponential, standard, 2nd Order",NaN,-177.974960,...,25122.68164,18373332.0,501,9246.724609,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,80.815497,1,85.325031
3,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,4,3000,"Exponential, standard, 2nd Order",NaN,-175.414535,...,8832.575195,19183462.0,501,9447.616211,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,88.345520,1,85.325031
4,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,5,4000,"Exponential, standard, 2nd Order",NaN,-185.801666,...,9454.709961,20073086.0,501,9661.617188,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,95.655040,1,85.325031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,6/17/2022,RK/N2,2022-06-17/RK/N2,2022_06_17_0006.abf,IN_0,16,3750,"Exponential, standard, 2nd Order",NaN,-153.385452,...,14187.65039,4637358.0,1001,2599.469482,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,58.778454,94,61.561810
1893,6/17/2022,RK/N2,2022-06-17/RK/N2,2022_06_17_0006.abf,IN_0,17,4000,"Exponential, standard, 2nd Order",NaN,-121.140343,...,17520.95117,4843433.5,1001,2651.708984,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,70.646057,94,61.561810
1894,6/17/2022,RK/N2,2022-06-17/RK/N2,2022_06_17_0006.abf,IN_0,18,4250,"Exponential, standard, 2nd Order",NaN,-131.036804,...,26666.89453,4864624.5,1001,2731.665527,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,62.076161,94,61.561810
1895,6/17/2022,RK/N2,2022-06-17/RK/N2,2022_06_17_0006.abf,IN_0,19,4500,"Exponential, standard, 2nd Order",NaN,-424.984985,...,30429.79883,4946642.0,1001,2696.283447,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,56.271429,94,61.561810


In [132]:
Med_Cp = Med_Cp.drop_duplicates('NeuronID')
Med_Cp

,Date,Neuron,NeuronID,FileName,Signal,Trace,Trace start,Fit function,Best number,A1,...,Sum of squared error,Sum of data squared,Points fitted,Fitted area,Results,File path,Cell Type,Capacitance (pF),Group,Median Capacitance (pF)
0,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-172.285049,...,8986.720703,18011766.0,501,9140.280273,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,90.944294,1,85.325031
17,9/22/2020,N3,2020-09-22/RK/N3,20922003.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-158.115250,...,14026.77637,15284945.0,501,8403.684570,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,95.445649,2,122.553061
37,9/22/2020,N5,2020-09-22/RK/N5,20922000.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-203.023071,...,15994.16602,13617701.0,501,7860.564453,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,41.674380,3,44.631994
57,9/23/2020,N1,2020-09-23/RK/N1,20923000_yy.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-265.953400,...,76810.65625,23959238.0,1001,7246.534668,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,55.508992,4,61.059604
77,9/23/2020,N2,2020-09-23/RK/N2,20923013.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-110.314995,...,60376.20313,27249036.0,1001,7998.471680,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,55.967704,5,44.230337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,6/17/2022,KM/N2,2022-06-17/KM/N2,22617009.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-72.792923,...,8094.288574,1822107.5,1001,1725.867188,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,20.044732,90,19.998048
1817,6/17/2022,KM/N3,2022-06-17/KM/N3,22617018.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-188.953445,...,15067.87891,8850166.0,1001,3973.163330,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,32.276580,91,32.575917
1837,6/17/2022,KM/N4,2022-06-17/KM/N4,22617027.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-112.238563,...,38035.36328,49190976.0,1001,10851.819340,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,97.049838,92,88.705139
1857,6/17/2022,KM/N5,2022-06-17/KM/N5,22617036.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-133.200729,...,19871.26367,3945184.0,1001,2457.501465,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,38.005841,93,36.162779


In [133]:
Med_Cp = Med_Cp[~Med_Cp.NeuronID.str.contains("2022-06-15/KM/N4")]

In [134]:
#Add Region column to DataFrame
regionlist = []

for i in range(len(Med_Cp)):
    
    if Med_Cp['NeuronID'].iloc[i] in list(ID_Region.keys()):
        regionlist.append(ID_Region[Med_Cp['NeuronID'].iloc[i]])
        
    else:
        regionlist.append('Nan')

Med_Cp['Region'] = regionlist

C:\Users\Ryan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [136]:
#Add Cell Type column to DataFrame
ctlist = []

for i in range(len(Med_Cp)):
    
    if Med_Cp['NeuronID'].iloc[i] in list(ID_CellType.keys()):
        ctlist.append(ID_CellType[Med_Cp['NeuronID'].iloc[i]])
        
    else:
        ctlist.append('NaN')

Med_Cp['Cell Type'] = ctlist

C:\Users\Ryan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [137]:
Med_Cp

,Date,Neuron,NeuronID,FileName,Signal,Trace,Trace start,Fit function,Best number,A1,...,Sum of data squared,Points fitted,Fitted area,Results,File path,Cell Type,Capacitance (pF),Group,Median Capacitance (pF),Region
0,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-172.285049,...,18011766.0,501,9140.280273,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,90.944294,1,85.325031,FF
17,9/22/2020,N3,2020-09-22/RK/N3,20922003.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-158.115250,...,15284945.0,501,8403.684570,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,95.445649,2,122.553061,vZI
37,9/22/2020,N5,2020-09-22/RK/N5,20922000.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-203.023071,...,13617701.0,501,7860.564453,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,41.674380,3,44.631994,vZI
57,9/23/2020,N1,2020-09-23/RK/N1,20923000_yy.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-265.953400,...,23959238.0,1001,7246.534668,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,55.508992,4,61.059604,vZI
77,9/23/2020,N2,2020-09-23/RK/N2,20923013.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-110.314995,...,27249036.0,1001,7998.471680,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,55.967704,5,44.230337,vZI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,6/17/2022,KM/N2,2022-06-17/KM/N2,22617009.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-72.792923,...,1822107.5,1001,1725.867188,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,20.044732,90,19.998048,d/vZI Border
1817,6/17/2022,KM/N3,2022-06-17/KM/N3,22617018.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-188.953445,...,8850166.0,1001,3973.163330,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,32.276580,91,32.575917,dZI
1837,6/17/2022,KM/N4,2022-06-17/KM/N4,22617027.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-112.238563,...,49190976.0,1001,10851.819340,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,97.049838,92,88.705139,dZI
1857,6/17/2022,KM/N5,2022-06-17/KM/N5,22617036.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-133.200729,...,3945184.0,1001,2457.501465,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,38.005841,93,36.162779,dZI


In [139]:
Med_Cp = Med_Cp.reset_index().drop(['index'], axis=1)
Med_Cp

,Date,Neuron,NeuronID,FileName,Signal,Trace,Trace start,Fit function,Best number,A1,...,Sum of data squared,Points fitted,Fitted area,Results,File path,Cell Type,Capacitance (pF),Group,Median Capacitance (pF),Region
0,9/22/2020,N2,2020-09-22/RK/N2,20922001_yy.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-172.285049,...,18011766.0,501,9140.280273,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,90.944294,1,85.325031,FF
1,9/22/2020,N3,2020-09-22/RK/N3,20922003.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-158.115250,...,15284945.0,501,8403.684570,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,95.445649,2,122.553061,vZI
2,9/22/2020,N5,2020-09-22/RK/N5,20922000.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-203.023071,...,13617701.0,501,7860.564453,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,41.674380,3,44.631994,vZI
3,9/23/2020,N1,2020-09-23/RK/N1,20923000_yy.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-265.953400,...,23959238.0,1001,7246.534668,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,55.508992,4,61.059604,vZI
4,9/23/2020,N2,2020-09-23/RK/N2,20923013.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-110.314995,...,27249036.0,1001,7998.471680,Successful fit.,Z:\RKDATA\PatchData\ZI\Spp1-Cre\AAV9-DIO-mCher...,Spp1,55.967704,5,44.230337,vZI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,6/17/2022,KM/N2,2022-06-17/KM/N2,22617009.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-72.792923,...,1822107.5,1001,1725.867188,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,20.044732,90,19.998048,d/vZI Border
91,6/17/2022,KM/N3,2022-06-17/KM/N3,22617018.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-188.953445,...,8850166.0,1001,3973.163330,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,32.276580,91,32.575917,dZI
92,6/17/2022,KM/N4,2022-06-17/KM/N4,22617027.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-112.238563,...,49190976.0,1001,10851.819340,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,97.049838,92,88.705139,dZI
93,6/17/2022,KM/N5,2022-06-17/KM/N5,22617036.abf,IN_0,1,0,"Exponential, standard, 2nd Order",NaN,-133.200729,...,3945184.0,1001,2457.501465,Successful fit.,W:\RKDATA\PatchData\ZIpatch\Slc30a3-Cre\AAV9-D...,Slc30a3,38.005841,93,36.162779,dZI


In [140]:
Med_Cp.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_MedCp_07102022.csv')

# Rm Data------------------------------------------------------------------------------------------

In [2]:
# Read In most updated version of DF to TIDY up
ZI_Ra_Rm_DF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/FinalVersions/2020-2022_RaRm_Sweepwise_07112022.csv')

In [168]:
ZI_Ra_Rm_DF[ZI_Ra_Rm_DF['Date'] == '9/22/2020']['Filepath'].tolist()

['/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre/AAV9-DIO-mCherry/2020/092220/RK/N2/20922008.abf',
 '/Volumes

In [95]:
ZI_Ra_Rm_DF[ZI_Ra_Rm_DF['NeuronID'] == '2021-06-17/KM/N1']

,Unnamed: 0,Unnamed: 0.1,level_0,index,Cell Type,Date,Mouse,Neuron,Filename,Protocol,NeuronID,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms),Filepath,Region
1876,1971,368,368,0,Spp1,6/17/2021,,KM/N1,21617000.abf,Membrane Test,2021-06-17/KM/N1,1,-14.073484,13.385622,862.333944,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1877,1976,388,388,0,Spp1,6/17/2021,,KM/N1,21617008.abf,Membrane Test,2021-06-17/KM/N1,1,-427.419040,8.815238,63.454551,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1878,1970,369,369,1,Spp1,6/17/2021,,KM/N1,21617000.abf,Membrane Test,2021-06-17/KM/N1,2,-14.379394,13.571903,888.098772,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1879,1977,389,389,1,Spp1,6/17/2021,,KM/N1,21617008.abf,Membrane Test,2021-06-17/KM/N1,2,-419.420000,8.933480,64.583331,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1880,1969,370,370,2,Spp1,6/17/2021,,KM/N1,21617000.abf,Membrane Test,2021-06-17/KM/N1,3,-15.546263,13.484775,786.912961,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1881,1998,390,390,2,Spp1,6/17/2021,,KM/N1,21617008.abf,Membrane Test,2021-06-17/KM/N1,3,-421.204440,8.888890,65.794216,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1882,1968,371,371,3,Spp1,6/17/2021,,KM/N1,21617000.abf,Membrane Test,2021-06-17/KM/N1,4,-16.842894,13.451561,872.222400,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1883,1997,391,391,3,Spp1,6/17/2021,,KM/N1,21617008.abf,Membrane Test,2021-06-17/KM/N1,4,-424.064420,8.863883,63.683882,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1884,1967,372,372,4,Spp1,6/17/2021,,KM/N1,21617000.abf,Membrane Test,2021-06-17/KM/N1,5,-21.064695,13.320326,787.586405,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI
1885,1996,392,392,4,Spp1,6/17/2021,,KM/N1,21617008.abf,Membrane Test,2021-06-17/KM/N1,5,-426.383150,8.784988,66.199010,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,vZI


In [3]:
set(ZI_Ra_Rm_DF['NeuronID'])

{'2020-09-22/RK/N2',
 '2020-09-22/RK/N3',
 '2020-09-22/RK/N5',
 '2020-09-23/KM/N1',
 '2020-09-23/KM/N2',
 '2020-09-23/KM/N3',
 '2020-09-23/RK/N1',
 '2020-09-23/RK/N2',
 '2020-09-23/RK/N3',
 '2020-09-23/RK/N4',
 '2020-09-23/RK/N5',
 '2020-09-23/RK/N6',
 '2020-09-24/KM/N1',
 '2020-09-24/KM/N2',
 '2020-09-24/KM/N3',
 '2020-09-24/KM/N4',
 '2020-09-24/KM/N5',
 '2020-09-24/KM/N6',
 '2020-10-01/KM/N1',
 '2020-10-01/KM/N2',
 '2020-10-01/KM/N3',
 '2020-10-01/KM/N4',
 '2020-10-01/KM/N5',
 '2020-10-01/KM/N6',
 '2020-10-01/RK/N1',
 '2020-10-01/RK/N2',
 '2020-10-01/RK/N3',
 '2020-10-01/RK/N4',
 '2020-10-01/RK/N5',
 '2020-10-01/RK/N6',
 '2020-10-02/KM/N1',
 '2020-10-02/KM/N2',
 '2020-10-02/KM/N3',
 '2020-10-02/KM/N4',
 '2020-10-02/KM/N5',
 '2020-10-02/RK/N1',
 '2020-10-02/RK/N2',
 '2020-10-02/RK/N3',
 '2021-06-15/KM/N1',
 '2021-06-17/KM/N1',
 '2021-06-17/KM/N2',
 '2021-06-17/KM/N3',
 '2021-06-21/KM/N1',
 '2021-06-21/KM/N2',
 '2021-06-21/KM/N3',
 '2021-06-21/KM/N4',
 '2021-06-21/KM/N5',
 '2021-06-21/

In [7]:
#Add Region column to DataFrame
regionlist = []

for i in range(len(ZI_Ra_Rm_DF)):
    
    if ZI_Ra_Rm_DF['NeuronID'].iloc[i] in list(ID_Region.keys()):
        regionlist.append(ID_Region[ZI_Ra_Rm_DF['NeuronID'].iloc[i]])
        
    else:
        regionlist.append('NaN')

ZI_Ra_Rm_DF['Region'] = regionlist

In [8]:
set(ZI_Ra_Rm_DF['Region'])

{'Ambiguous',
 'FF',
 'NaN',
 'PIL',
 'SNR',
 'SNr',
 'STN',
 'cZI',
 'd/vZI',
 'd/vZI Border',
 'dZI',
 'rZI',
 'vLGN',
 'vZI'}

In [9]:
ZI_Ra_Rm_DF = ZI_Ra_Rm_DF[~ZI_Ra_Rm_DF.Region.str.contains("NaN")]

In [153]:
ZI_Ra_Rm_DF.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/FinalVersions/2020-2022_RaRm_Sweepwise_07112022.csv')

In [11]:
ZI_Ra_Rm_DF = ZI_Ra_Rm_DF.sort_values(['NeuronID', 'Filename', 'Sweep Number'])
ZI_Ra_Rm_DF

,Unnamed: 0,Unnamed: 0.1,level_0,index,Cell Type,Date,Mouse,Neuron,Filename,Protocol,NeuronID,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms),Filepath,Region
0,2,1818,1818,0,Spp1,9/22/2020,,RK/N2,20922008.abf,Membrane Test,2020-09-22/RK/N2,1,-53.248592,15.648520,211.495126,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,FF
1,3,1819,1819,1,Spp1,9/22/2020,,RK/N2,20922008.abf,Membrane Test,2020-09-22/RK/N2,2,-56.303406,15.611243,215.503805,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,FF
2,4,1820,1820,2,Spp1,9/22/2020,,RK/N2,20922008.abf,Membrane Test,2020-09-22/RK/N2,3,-57.454838,15.493144,209.862927,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,FF
3,5,1821,1821,3,Spp1,9/22/2020,,RK/N2,20922008.abf,Membrane Test,2020-09-22/RK/N2,4,-61.058960,15.596383,220.766971,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,FF
4,6,1822,1822,4,Spp1,9/22/2020,,RK/N2,20922008.abf,Membrane Test,2020-09-22/RK/N2,5,-59.450380,15.434762,214.246986,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,FF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5034,4992,5717,712,15,slc30a3,6/17/2022,,RK/N2,_17_0013.abf,Membrane Test,2022-06-17/RK/N2,16,56.334103,17.496798,152.303345,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,dZI
5036,4991,5718,713,16,slc30a3,6/17/2022,,RK/N2,_17_0013.abf,Membrane Test,2022-06-17/RK/N2,17,56.741330,17.519249,141.955824,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,dZI
5038,4990,5719,714,17,slc30a3,6/17/2022,,RK/N2,_17_0013.abf,Membrane Test,2022-06-17/RK/N2,18,54.400387,17.485594,152.645886,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,dZI
5040,4989,5720,715,18,slc30a3,6/17/2022,,RK/N2,_17_0013.abf,Membrane Test,2022-06-17/RK/N2,19,52.847164,17.870857,155.203856,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,dZI


In [12]:
##Check the output of this cell, and make sure the Filename column is chronologically ordered

AveRa_Rm = ZI_Ra_Rm_DF.groupby(['NeuronID','Filename', 'Cell Type', 'Date']).mean()

In [15]:
test = AveRa_Rm.head(30)

test

,NeuronID,Filename,Cell Type,Date,Unnamed: 0,Unnamed: 0.1,level_0,index,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
0,2020-09-22/RK/N2,20922008.abf,Spp1,9/22/2020,49.560000,1842.5,1842.5,24.5,25.5,-61.024670,15.230345,228.634506
1,2020-09-22/RK/N3,20922000.abf,Spp1,9/22/2020,165.333333,1696.0,1696.0,1.0,2.0,-183.126430,12.868974,229.786248
2,2020-09-22/RK/N3,20922003.abf,Spp1,9/22/2020,129.000000,1707.5,1707.5,9.5,10.5,-109.254822,14.486928,180.295425
3,2020-09-22/RK/N3,20922010.abf,Spp1,9/22/2020,105.800000,1727.5,1727.5,9.5,10.5,-103.687378,13.813489,158.589378
4,2020-09-22/RK/N3,20922011.abf,Spp1,9/22/2020,160.900000,1747.5,1747.5,9.5,10.5,-100.747681,12.188323,157.830345
5,2020-09-22/RK/N5,20922000.abf,Spp1,9/22/2020,161.800000,1767.5,1767.5,9.5,10.5,-104.032883,11.243232,238.130767
6,2020-09-22/RK/N5,20922007.abf,Spp1,9/22/2020,108.300000,1787.5,1787.5,9.5,10.5,-46.969498,19.764985,480.946385
7,2020-09-22/RK/N5,20922011.abf,Spp1,9/22/2020,30.450000,1807.5,1807.5,9.5,10.5,0.325439,16.245239,392.665594
8,2020-09-23/KM/N1,20923000.abf,Spp1,9/23/2020,255.050000,2194.5,2194.5,9.5,10.5,-227.711209,5.599573,96.675115
9,2020-09-23/KM/N2,20923008.abf,Spp1,9/23/2020,365.300000,2234.5,2234.5,9.5,10.5,-153.350571,8.913073,233.206144


In [14]:
AveRa_Rm = AveRa_Rm.reset_index(level=['NeuronID','Filename','Cell Type','Date'])
AveRa_Rm

,NeuronID,Filename,Cell Type,Date,Unnamed: 0,Unnamed: 0.1,level_0,index,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
0,2020-09-22/RK/N2,20922008.abf,Spp1,9/22/2020,49.560000,1842.5,1842.5,24.5,25.5,-61.024670,15.230345,228.634506
1,2020-09-22/RK/N3,20922000.abf,Spp1,9/22/2020,165.333333,1696.0,1696.0,1.0,2.0,-183.126430,12.868974,229.786248
2,2020-09-22/RK/N3,20922003.abf,Spp1,9/22/2020,129.000000,1707.5,1707.5,9.5,10.5,-109.254822,14.486928,180.295425
3,2020-09-22/RK/N3,20922010.abf,Spp1,9/22/2020,105.800000,1727.5,1727.5,9.5,10.5,-103.687378,13.813489,158.589378
4,2020-09-22/RK/N3,20922011.abf,Spp1,9/22/2020,160.900000,1747.5,1747.5,9.5,10.5,-100.747681,12.188323,157.830345
...,...,...,...,...,...,...,...,...,...,...,...,...
247,2022-06-17/KM/N4,22617034.abf,slc30a3,6/17/2022,5098.100000,5602.5,597.5,9.5,10.5,38.438986,14.568101,129.333016
248,2022-06-17/KM/N5,22617036.abf,slc30a3,6/17/2022,4956.350000,5662.5,657.5,9.5,10.5,-26.302463,12.439261,799.607800
249,2022-06-17/KM/N5,22617043.abf,slc30a3,6/17/2022,4963.050000,5642.5,637.5,9.5,10.5,39.438796,14.432702,256.363932
250,2022-06-17/RK/N2,_17_0006.abf,slc30a3,6/17/2022,4934.950000,5691.5,686.5,9.5,10.5,-24.246537,15.647604,858.554385


In [109]:
AveRa_Rm[AveRa_Rm['NeuronID'] == '2021-06-17/KM/N1']

,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
94,2021-06-17/KM/N1,21617000.abf,Spp1,6/17/2021,10.5,-23.403910,13.247047,988.751869
95,2021-06-17/KM/N1,21617008.abf,Spp1,6/17/2021,10.5,-422.184849,8.863808,65.961630


In [21]:
AveRa_Rm = AveRa_Rm.drop('level_0', axis=1)

In [24]:
AveRa_Rm = AveRa_Rm.sort_values(['NeuronID', 'Filename'], ascending=True)
AveRa_Rm

,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
0,2020-09-22/RK/N2,20922008.abf,Spp1,9/22/2020,25.5,-61.024670,15.230345,228.634506
1,2020-09-22/RK/N3,20922000.abf,Spp1,9/22/2020,2.0,-183.126430,12.868974,229.786248
2,2020-09-22/RK/N3,20922003.abf,Spp1,9/22/2020,10.5,-109.254822,14.486928,180.295425
3,2020-09-22/RK/N3,20922010.abf,Spp1,9/22/2020,10.5,-103.687378,13.813489,158.589378
4,2020-09-22/RK/N3,20922011.abf,Spp1,9/22/2020,10.5,-100.747681,12.188323,157.830345
...,...,...,...,...,...,...,...,...
247,2022-06-17/KM/N4,22617034.abf,slc30a3,6/17/2022,10.5,38.438986,14.568101,129.333016
248,2022-06-17/KM/N5,22617036.abf,slc30a3,6/17/2022,10.5,-26.302463,12.439261,799.607800
249,2022-06-17/KM/N5,22617043.abf,slc30a3,6/17/2022,10.5,39.438796,14.432702,256.363932
250,2022-06-17/RK/N2,_17_0006.abf,slc30a3,6/17/2022,10.5,-24.246537,15.647604,858.554385


In [29]:
AveRa_Rm[AveRa_Rm['NeuronID'] == '2020-09-23/RK/N1']

,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
13,2020-09-23/RK/N1,20923010.abf,Spp1,9/23/2020,10.5,82.950563,11.657706,143.42937


In [27]:
AveRa_Rm[AveRa_Rm['NeuronID'] == '2020-09-23/RK/N5']

,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
21,2020-09-23/RK/N5,20923000.abf,Spp1,9/23/2020,10.5,-150.371223,9.312562,166.129638
22,2020-09-23/RK/N5,20923007.abf,Spp1,9/23/2020,10.5,-67.257408,10.595658,171.614697


In [30]:
AveRa_Rm_Start = AveRa_Rm.drop_duplicates('NeuronID').reset_index()
AveRa_Rm_Start[AveRa_Rm_Start['NeuronID'] == '2020-09-23/RK/N5']

,index,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
10,21,2020-09-23/RK/N5,20923000.abf,Spp1,9/23/2020,10.5,-150.371223,9.312562,166.129638


In [31]:
AveRa_Rm_reverse = AveRa_Rm.sort_values(['NeuronID', 'Filename'], ascending = False)
AveRa_Rm_reverse[AveRa_Rm_reverse['NeuronID'] == '2020-09-23/RK/N5']

,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
22,2020-09-23/RK/N5,20923007.abf,Spp1,9/23/2020,10.5,-67.257408,10.595658,171.614697
21,2020-09-23/RK/N5,20923000.abf,Spp1,9/23/2020,10.5,-150.371223,9.312562,166.129638


In [143]:
Test = AveRa_Rm_reverse[AveRa_Rm_reverse['Date'] == '8/4/2021']

In [144]:
Test

,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
149,2021-08-04/KM/N3,21804023.abf,Spp1,8/4/2021,10.5,-33.297115,22.486148,246.117150
148,2021-08-04/KM/N3,21804017.abf,Spp1,8/4/2021,10.5,-72.953679,16.891537,266.599695
147,2021-08-04/KM/N2,21804015.abf,Spp1,8/4/2021,10.5,-60.709517,23.724542,379.058409
146,2021-08-04/KM/N2,21804009.abf,Spp1,8/4/2021,10.5,-141.886635,21.994500,235.592956
145,2021-08-04/KM/N1,21804008.abf,Spp1,8/4/2021,10.5,-65.175091,15.091166,322.994386
144,2021-08-04/KM/N1,21804006.abf,Spp1,8/4/2021,10.5,-50.793294,52.038930,358.591700
143,2021-08-04/KM/N1,21804000.abf,Spp1,8/4/2021,10.5,-116.779753,17.138603,279.861587


In [32]:
AveRa_Rm_End = AveRa_Rm_reverse.drop_duplicates('NeuronID').reset_index()
AveRa_Rm_End

,index,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
0,251,2022-06-17/RK/N2,_17_0013.abf,slc30a3,6/17/2022,10.5,60.418396,17.293256,142.029074
1,249,2022-06-17/KM/N5,22617043.abf,slc30a3,6/17/2022,10.5,39.438796,14.432702,256.363932
2,247,2022-06-17/KM/N4,22617034.abf,slc30a3,6/17/2022,10.5,38.438986,14.568101,129.333016
3,245,2022-06-17/KM/N3,22617025.abf,slc30a3,6/17/2022,10.5,116.846428,23.915203,103.413292
4,243,2022-06-17/KM/N2,22617016.abf,slc30a3,6/17/2022,10.5,-0.683002,19.744380,1175.063540
...,...,...,...,...,...,...,...,...,...
112,10,2020-09-23/KM/N2,20923015.abf,Spp1,9/23/2020,10.5,-157.685773,12.554315,93.953947
113,8,2020-09-23/KM/N1,20923000.abf,Spp1,9/23/2020,10.5,-227.711209,5.599573,96.675115
114,7,2020-09-22/RK/N5,20922011.abf,Spp1,9/22/2020,10.5,0.325439,16.245239,392.665594
115,4,2020-09-22/RK/N3,20922011.abf,Spp1,9/22/2020,10.5,-100.747681,12.188323,157.830345


In [33]:
AveRa_Rm_End = AveRa_Rm_End.sort_values('NeuronID', ascending=True)
AveRa_Rm_End

,index,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
116,0,2020-09-22/RK/N2,20922008.abf,Spp1,9/22/2020,25.5,-61.024670,15.230345,228.634506
115,4,2020-09-22/RK/N3,20922011.abf,Spp1,9/22/2020,10.5,-100.747681,12.188323,157.830345
114,7,2020-09-22/RK/N5,20922011.abf,Spp1,9/22/2020,10.5,0.325439,16.245239,392.665594
113,8,2020-09-23/KM/N1,20923000.abf,Spp1,9/23/2020,10.5,-227.711209,5.599573,96.675115
112,10,2020-09-23/KM/N2,20923015.abf,Spp1,9/23/2020,10.5,-157.685773,12.554315,93.953947
...,...,...,...,...,...,...,...,...,...
4,243,2022-06-17/KM/N2,22617016.abf,slc30a3,6/17/2022,10.5,-0.683002,19.744380,1175.063540
3,245,2022-06-17/KM/N3,22617025.abf,slc30a3,6/17/2022,10.5,116.846428,23.915203,103.413292
2,247,2022-06-17/KM/N4,22617034.abf,slc30a3,6/17/2022,10.5,38.438986,14.568101,129.333016
1,249,2022-06-17/KM/N5,22617043.abf,slc30a3,6/17/2022,10.5,39.438796,14.432702,256.363932


In [35]:
AveRa_Rm_End[AveRa_Rm_End['NeuronID'] == '2020-09-23/RK/N5']

,index,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms)
106,22,2020-09-23/RK/N5,20923007.abf,Spp1,9/23/2020,10.5,-67.257408,10.595658,171.614697


In [36]:
AveRa_Rm_Start['End Holding Current (pA)'] = AveRa_Rm_End['Holding Current (pA)'].values
AveRa_Rm_Start['End Average Ra (MOhms)'] = AveRa_Rm_End['Sweep Ra (MOhms)'].values
AveRa_Rm_Start['End Average Rm (MOhms)'] = AveRa_Rm_End['Sweep Rm (MOhms)'].values
AveRa_Rm_Start

,index,NeuronID,Filename,Cell Type,Date,Sweep Number,Holding Current (pA),Sweep Ra (MOhms),Sweep Rm (MOhms),End Holding Current (pA),End Average Ra (MOhms),End Average Rm (MOhms)
0,0,2020-09-22/RK/N2,20922008.abf,Spp1,9/22/2020,25.5,-61.024670,15.230345,228.634506,-61.024670,15.230345,228.634506
1,1,2020-09-22/RK/N3,20922000.abf,Spp1,9/22/2020,2.0,-183.126430,12.868974,229.786248,-100.747681,12.188323,157.830345
2,5,2020-09-22/RK/N5,20922000.abf,Spp1,9/22/2020,10.5,-104.032883,11.243232,238.130767,0.325439,16.245239,392.665594
3,8,2020-09-23/KM/N1,20923000.abf,Spp1,9/23/2020,10.5,-227.711209,5.599573,96.675115,-227.711209,5.599573,96.675115
4,9,2020-09-23/KM/N2,20923008.abf,Spp1,9/23/2020,10.5,-153.350571,8.913073,233.206144,-157.685773,12.554315,93.953947
...,...,...,...,...,...,...,...,...,...,...,...,...
112,242,2022-06-17/KM/N2,22617009.abf,slc30a3,6/17/2022,10.5,-20.978098,17.996699,1170.193412,-0.683002,19.744380,1175.063540
113,244,2022-06-17/KM/N3,22617018.abf,slc30a3,6/17/2022,10.5,-46.967385,10.368155,431.474213,116.846428,23.915203,103.413292
114,246,2022-06-17/KM/N4,22617027.abf,slc30a3,6/17/2022,10.5,-160.728214,11.656967,255.193501,38.438986,14.568101,129.333016
115,248,2022-06-17/KM/N5,22617036.abf,slc30a3,6/17/2022,10.5,-26.302463,12.439261,799.607800,39.438796,14.432702,256.363932


In [37]:
AveRa_Rm_Both = AveRa_Rm_Start.rename(columns={"Holding Current (pA)": "Start Holding Current (pA)","Sweep Ra (MOhms)": "Start Average Ra (MOhms)", "Sweep Rm (MOhms)": "Start Average Rm (MOhms)"})
AveRa_Rm_Both

,index,NeuronID,Filename,Cell Type,Date,Sweep Number,Start Holding Current (pA),Start Average Ra (MOhms),Start Average Rm (MOhms),End Holding Current (pA),End Average Ra (MOhms),End Average Rm (MOhms)
0,0,2020-09-22/RK/N2,20922008.abf,Spp1,9/22/2020,25.5,-61.024670,15.230345,228.634506,-61.024670,15.230345,228.634506
1,1,2020-09-22/RK/N3,20922000.abf,Spp1,9/22/2020,2.0,-183.126430,12.868974,229.786248,-100.747681,12.188323,157.830345
2,5,2020-09-22/RK/N5,20922000.abf,Spp1,9/22/2020,10.5,-104.032883,11.243232,238.130767,0.325439,16.245239,392.665594
3,8,2020-09-23/KM/N1,20923000.abf,Spp1,9/23/2020,10.5,-227.711209,5.599573,96.675115,-227.711209,5.599573,96.675115
4,9,2020-09-23/KM/N2,20923008.abf,Spp1,9/23/2020,10.5,-153.350571,8.913073,233.206144,-157.685773,12.554315,93.953947
...,...,...,...,...,...,...,...,...,...,...,...,...
112,242,2022-06-17/KM/N2,22617009.abf,slc30a3,6/17/2022,10.5,-20.978098,17.996699,1170.193412,-0.683002,19.744380,1175.063540
113,244,2022-06-17/KM/N3,22617018.abf,slc30a3,6/17/2022,10.5,-46.967385,10.368155,431.474213,116.846428,23.915203,103.413292
114,246,2022-06-17/KM/N4,22617027.abf,slc30a3,6/17/2022,10.5,-160.728214,11.656967,255.193501,38.438986,14.568101,129.333016
115,248,2022-06-17/KM/N5,22617036.abf,slc30a3,6/17/2022,10.5,-26.302463,12.439261,799.607800,39.438796,14.432702,256.363932


In [43]:
AveRa_Rm_Both[AveRa_Rm_Both['NeuronID'] == '2020-09-23/RK/N1']

,index,NeuronID,Filename,Cell Type,Date,Sweep Number,Start Holding Current (pA),Start Average Ra (MOhms),Start Average Rm (MOhms),End Holding Current (pA),End Average Ra (MOhms),End Average Rm (MOhms)
6,13,2020-09-23/RK/N1,20923010.abf,Spp1,9/23/2020,10.5,82.950563,11.657706,143.42937,82.950563,11.657706,143.42937


In [151]:
#Add Region column to DataFrame
regionlist = []

for i in range(len(AveRa_Rm_Start)):
    
    if AveRa_Rm_Start['NeuronID'].iloc[i] in list(ID_Region.keys()):
        regionlist.append(ID_Region[AveRa_Rm_Start['NeuronID'].iloc[i]])
        
    else:
        regionlist.append('NaN')

AveRa_Rm_Start['Region'] = regionlist

In [44]:
AveRa_Rm_Both.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_StartEnd_AveRaRm_08102022.csv')

In [153]:
test = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_StartEnd_AveRaRm_07142022.csv')

In [158]:
AveRa_Rm_Both.sort_values('End Holding Current (pA)', ascending = False).head(30)

,index,NeuronID,Filename,Cell Type,Date,Sweep Number,Start Holding Current (pA),Start Average Ra (MOhms),Start Average Rm (MOhms),End Holding Current (pA),End Average Ra (MOhms),End Average Rm (MOhms)
52,120,2021-06-28/KM/N3,21628014.abf,Spp1,6/28/2021,10.5,-55.905073,15.269852,270.141301,215.468520,35.559355,65.164758
113,244,2022-06-17/KM/N3,22617018.abf,slc30a3,6/17/2022,10.5,-46.967385,10.368155,431.474213,116.846428,23.915203,103.413292
95,206,2021-12-22/KM/N3,21d22016.abf,Nos1,12/22/2021,10.5,-49.370851,14.152930,442.164805,90.107512,25.546812,104.700272
6,21,2020-09-23/RK/N5,20923000.abf,Spp1,9/23/2020,10.5,-150.371223,9.312562,166.129638,82.950563,11.657706,143.429370
108,232,2022-06-15/KM/N1,22615012.abf,slc30a3,6/15/2022,10.5,-21.426835,20.603393,955.505143,77.667947,20.623547,111.660811
77,170,2021-09-10/KM/N3,21910018.abf,Nos1,9/10/2021,10.5,-75.270025,13.239490,385.719563,66.147552,24.421159,104.691456
116,250,2022-06-17/RK/N2,_17_0006.abf,slc30a3,6/17/2022,10.5,-24.246537,15.647604,858.554385,60.418396,17.293256,142.029074
54,124,2021-06-28/KM/N5,21628029.abf,Spp1,6/28/2021,10.5,-82.424942,12.534685,385.322546,58.604816,15.896987,112.542115
80,176,2021-09-13/KM/N1,21913000.abf,Nos1,9/13/2021,10.5,-33.638106,18.652330,855.603893,58.532369,24.027738,179.366869
70,156,2021-08-06/KM/N3,21806016.abf,Spp1,8/6/2021,10.5,-307.588984,10.237426,142.549723,54.603726,16.736651,60.367614


In [162]:
AveRa_Rm_Both.to_csv('C:/Users/Ryan/Desktop/2020-2022_StartEnd_AveRaRm_08042022.csv')

# RMP Data------------------------------------------------------------------------------------------

In [45]:
# Read In most updated version of DF to TIDY up
ZI_RMP_DF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_RMP_07082022.csv')

In [46]:
ZI_RMP_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              167 non-null    int64  
 1   level_0                 167 non-null    int64  
 2   index                   167 non-null    int64  
 3   Cell Type               167 non-null    object 
 4   Date                    167 non-null    object 
 5   Mouse                   167 non-null    object 
 6   Neuron                  167 non-null    object 
 7   Filename                167 non-null    object 
 8   Protocol                167 non-null    object 
 9   Filepath                167 non-null    object 
 10  RMP 1                   167 non-null    float64
 11  Max Membrane Voltage 1  167 non-null    float64
 12  STDEV 1                 167 non-null    float64
 13  RMP 2                   167 non-null    float64
 14  Max Membrane Voltage 2  167 non-null    fl

In [49]:
ZI_RMP_DF = ZI_RMP_DF.drop(['index'], axis = 1)
ZI_RMP_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Cell Type               167 non-null    object 
 1   Date                    167 non-null    object 
 2   Mouse                   167 non-null    object 
 3   Neuron                  167 non-null    object 
 4   Filename                167 non-null    object 
 5   Protocol                167 non-null    object 
 6   Filepath                167 non-null    object 
 7   RMP 1                   167 non-null    float64
 8   Max Membrane Voltage 1  167 non-null    float64
 9   STDEV 1                 167 non-null    float64
 10  RMP 2                   167 non-null    float64
 11  Max Membrane Voltage 2  167 non-null    float64
 12  STDEV 2                 167 non-null    float64
 13  NeuronID                167 non-null    object 
dtypes: float64(6), object(8)
memory usage: 18.

In [98]:
ZI_RMP_DF = ZI_RMP_DF.replace('05.ab','RK/N2')

In [102]:
ZI_RMP_DF = ZI_RMP_DF.replace('2022-06-17/05.ab','2022-06-17/RK/N2')

In [104]:
#Add Region column to DataFrame
regionlist = []

for i in range(len(ZI_RMP_DF)):
    
    if ZI_RMP_DF['NeuronID'].iloc[i] in list(ID_Region.keys()):
        regionlist.append(ID_Region[ZI_RMP_DF['NeuronID'].iloc[i]])
        
    else:
        regionlist.append('Nan')

ZI_RMP_DF['Region'] = regionlist

In [344]:
ZI_RMP_DF = ZI_RMP_DF[~ZI_RMP_DF.NeuronID.str.contains("2022-06-17/RK/N3")]

In [100]:
set(ZI_RMP_DF['Region'])

{'Ambiguous',
 'FF',
 'Nan',
 'PIL',
 'SNR',
 'SNr',
 'STN',
 'cZI',
 'd/vZI',
 'd/vZI Border',
 'dZI',
 'vLGN',
 'vZI'}

In [107]:
ZI_RMP_DF[ZI_RMP_DF['Region'] == 'Nan']

,Cell Type,Date,Mouse,Neuron,Filename,Protocol,Filepath,RMP 1,Max Membrane Voltage 1,STDEV 1,RMP 2,Max Membrane Voltage 2,STDEV 2,NeuronID,Region


In [106]:
ZI_RMP_DF = ZI_RMP_DF[~ZI_RMP_DF.Region.str.contains("Nan")]

In [108]:
ZI_RMP_DF.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_RMP_07122022.csv')

# Voltage Sag Data--------------------------------------------------------------------------------------

In [71]:
# Read In most updated version of DF to TIDY up
ZI_vsag_DF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/SweepSpecific/FinalVersions/2020-2022_vsag_07082022.csv')

In [72]:
ZI_vsag_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           612 non-null    int64  
 1   level_0                              612 non-null    int64  
 2   index                                612 non-null    int64  
 3   Cell Type                            612 non-null    object 
 4   Date                                 612 non-null    object 
 5   Mouse                                612 non-null    object 
 6   Neuron                               612 non-null    object 
 7   Filename                             612 non-null    object 
 8   Protocol                             612 non-null    object 
 9   Filepath                             612 non-null    object 
 10  Sweep Number                         612 non-null    int64  
 11  Holding Current (pA)            

In [75]:
ZI_vsag_DF = ZI_vsag_DF.drop('index', axis = 1)

In [76]:
ZI_vsag_DFneuron = ZI_vsag_DF[(ZI_vsag_DF['Peak Negative Membrane Voltage (mV)'] >= -110) & (ZI_vsag_DF['Peak Negative Membrane Voltage (mV)'] <= -100)]

In [92]:
#Add Region column to DataFrame
regionlist = []

for i in range(len(ZI_vsag_DFneuron)):
    
    if ZI_vsag_DFneuron['NeuronID'].iloc[i] in list(ID_Region.keys()):
        regionlist.append(ID_Region[ZI_vsag_DFneuron['NeuronID'].iloc[i]])
        
    else:
        regionlist.append('Nan')

ZI_vsag_DFneuron['Region'] = regionlist

In [93]:
ZI_vsag_DFneuron

,Cell Type,Date,Mouse,Neuron,Filename,Protocol,Filepath,Sweep Number,Holding Current (pA),Current Step (pA),Pre-step Membrane Voltage (mV),Peak Negative Membrane Voltage (mV),Voltage Sag (mV),NeuronID,Region
4,Spp1,2020-09-22,,RK/N2,20922005.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,2,0.0,-150.0,-61.371710,-103.698730,12.408470,2020-09-22/RK/N2,FF
10,Spp1,2020-09-22,,RK/N5,20922015.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,2,0.0,-150.0,-59.462390,-108.825680,1.330231,2020-09-22/RK/N5,vZI
11,Spp1,2020-09-22,,RK/N5,20922015.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,1,0.0,-100.0,-52.483930,-100.799560,1.962601,2020-09-22/RK/N5,vZI
16,Spp1,2020-09-22,,RK/N5,20922009.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,2,0.0,-150.0,-63.348553,-108.184814,1.746925,2020-09-22/RK/N5,vZI
23,Spp1,2020-09-22,,RK/N3,20922008.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,5,0.0,-300.0,-54.247540,-100.921630,2.629417,2020-09-22/RK/N3,vZI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,slc30a3,2022-06-17,,KM/N2,22617012.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,0,0.0,-45.0,-58.603477,-104.034424,0.645027,2022-06-17/KM/N2,d/vZI Border
601,slc30a3,2022-06-17,,KM/N5,22617039.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,2,0.0,-100.0,-63.808243,-102.416990,1.995804,2022-06-17/KM/N5,dZI
603,slc30a3,2022-06-17,,KM/N3,22617021.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,4,0.0,-245.0,-65.080490,-104.949950,0.220161,2022-06-17/KM/N3,dZI
607,slc30a3,2022-06-17,,KM/N4,22617030.abf,IC rebound IF,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,2,0.0,-145.0,-64.830280,-102.661130,3.413963,2022-06-17/KM/N4,dZI


In [94]:
check = ZI_vsag_DFneuron[ZI_vsag_DFneuron['Region'] == 'Nan']
check

,Cell Type,Date,Mouse,Neuron,Filename,Protocol,Filepath,Sweep Number,Holding Current (pA),Current Step (pA),Pre-step Membrane Voltage (mV),Peak Negative Membrane Voltage (mV),Voltage Sag (mV),NeuronID,Region


In [95]:
ZI_vsag_DFneuron.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_Vsag_07122022.csv.csv')

In [163]:
ZI_vsag_DFneuron.sort_values(by=['NeuronID', 'Filename']
                             ,axis=0).to_csv('Z:/RKDATA/PatchData/ZI/CompiledAnalysis/csv/NeuronLevel/NeuronLevelZI_vsag_-100to-110mV.csv')